In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay,
    RocCurveDisplay, PrecisionRecallDisplay
)
from joblib import dump, load
import swifter
print("\nLoading SpaCy model...")
nlp = spacy.load("en_core_web_sm")
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return ' '.join(tokens)
def svo_structure_spacy(text):
    if not isinstance(text, str) or len(text) == 0:
        return 1
    doc = nlp(text)
    subjects = any(token.dep_ in ["nsubj", "nsubjpass"] for token in doc)
    objects = any(token.dep_ in ["dobj", "pobj"] for token in doc)
    verbs = any(token.pos_ == "VERB" for token in doc)
    return 0 if (subjects and objects and verbs) else 1
file_path = r"C:\Users\KIIT\Desktop\ML proj\Phishing_New.csv"
data = pd.read_csv(file_path)
print("Preprocessing and extracting features...")
data['Processed_Text'] = data['Email Text'].swifter.apply(preprocess_text)
data['SVO_Structure'] = data['Processed_Text'].swifter.apply(svo_structure_spacy)
data.to_csv(file_path, index=False)
vectorizer = TfidfVectorizer(max_features=3000)
X_vec = vectorizer.fit_transform(data['Processed_Text']).toarray()
X = np.column_stack((X_vec, data['SVO_Structure'].values))
y = data['Email Type'].apply(lambda x: 1 if x == 'Phishing Email' else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training model...")
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)
dump(model, 'phishing_model_spacy.pkl')
dump(vectorizer, 'vectorizer_spacy.pkl')
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("\nModel Evaluation:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()
plt.title("Confusion Matrix")
plt.show()
RocCurveDisplay.from_estimator(model, X_test, y_test, name="Logistic Regression")
plt.title("ROC Curve")
plt.grid(True)
plt.show()
PrecisionRecallDisplay.from_estimator(model, X_test, y_test, name="Logistic Regression")
plt.title("Precision-Recall Curve")
plt.grid(True)
plt.show()
def classify_email(email):
    model = load('phishing_model_spacy.pkl')
    vectorizer = load('vectorizer_spacy.pkl')
    processed_email = preprocess_text(email)
    svo_feature = np.array([svo_structure_spacy(processed_email)])
    email_vec = vectorizer.transform([processed_email]).toarray()
    email_features = np.column_stack((email_vec, svo_feature))
    prediction = model.predict(email_features)[0]
    result = "Phishing Email" if prediction == 1 else "Safe Email"
    print(f"\nEmail Classification Result: {result}")
print("\nTesting Real-Time Email Classification...")
classify_email("Congratulations! You have won $1,000,000. Click here to claim your prize.")
classify_email("Hey John, just checking in to confirm our meeting schedule for next week.")